# Getting Data API
Requesting data from Steam Web API.
The users IDs (SteamID) were obtained from a separate database. The IDs are used to get data about friend connections and owned games of each user on Steam gaming platform.

In [15]:
import requests
import json
import pandas as pd

In [ ]:
# Getting friend lists from Steam API
URL = 'https://api.steampowered.com/ISteamUser/GetFriendList/v0001/'

# Function Requesting data from API
def getdata_friendlist(steamid,URL):
    api_key = 'API Key' # this is your personal Steam API key
    key_string = '?' + 'key=' + api_key
    path = URL + key_string + '&steamid=' + steamid + '&relationship=friend'
    response = requests.get(url=path)
    return response.json()

# Reading SteamIDs from a file
filename = "...\players.csv"
df_players = pd.read_csv(filename, index_col=0)
steamids = df_players['PlayerID']

# Making dataframe of friend connections
filename= ".../friends.csv"
open(filename,'a')

df_friends = pd.DataFrame()
for i in range(len(steamids)):
    data = getdata_friendlist(str(steamids[i]),URL)
    # checking if the profile is public
    if data != {}:
        # checking if the friend list is public
        if data['friendslist']['friends'] != []:
            df = pd.io.json.json_normalize(data['friendslist']['friends'])
            df = df.drop('relationship', axis='columns')
            df.columns = ['friend_since','FriendID']
            df['PlayerID'] = steamids[i]
        else:
            df = pd.DataFrame({'friend_since':0,'FriendID':'none','PlayerID':steamids[i]}, index=[0])
    else:
        df = pd.DataFrame({'friend_since':0,'FriendID':'none','PlayerID':steamids[i]}, index=[0])
    # saving data to a file
    df.to_csv(filename,mode='a',header=False)
    # printing the progress of API requests
    if (i % 100 == 0):
        progress = round(i/len(steamids),4)*100
        print('%0.2f' % progress,'%, # of friends:',df.count()[0])
filename.close()

In [ ]:
# Getting the list of owned games from Steam API
URL = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'

# Function Requesting data from API
def getdata_ownedgames(steamid,URL):
    api_key = 'API Key' # this is your personal Steam API key
    key_string = '?' + 'key=' + api_key
    path = URL + key_string + '&steamid=' + steamid
    var = {'include_played_free_games':1}
    response = requests.get(url=path, params=var)
    return response.json()

# Reading SteamIDs from a file
filename = "...\players.csv"
df_players = pd.read_csv(filename, index_col=0)
steamids = df_players['PlayerID']

# Making dataframe of owned games
for i in range(len(steamids)):
    data = getdata_ownedgames(str(steamids[i]),URL)
    # saving data asa json file
    with open('.../SteamID_%d.json' % steamids[i],'w') as file:
        json.dump(data,file)
    # printing the progress of API requests
    if (i % 100 == 0):
        progress = round(i/len(steamids),4)*100
        try: print('%0.2f' % progress,'%, # of games:',data['response']['game_count'])
        except: print('%0.2f' % progress,'%, # of games:',0)